**Mínim eigenvalue de la transposta de la Identitat del subespai simètric**

In [1]:
import numpy as np
import math
from toqito.channels import partial_transpose
from toqito.perms import symmetric_projection

d=2
N=2

total_dimension = d**N #Dimensió de l'espai de Hilbert Global
dimensions = [d]*N #Dimensions dels espais de Hilbert locals

biggest_bipartition = int(math.floor(N/2)) #Respecte quants qudits es fa la bipartició més gran

symmetric_dimension = math.comb(d+N-1,N) #Dimensió del subespai simètric

IS = symmetric_projection(d,N) #Identitat del Subespai Simètric

symmetric_transpose_dimension = math.comb(d+biggest_bipartition-1,biggest_bipartition)*math.comb(d+N-biggest_bipartition-1,N-biggest_bipartition) #La transposada parcial de quelcom que viu en S(C^d ** \otimes N) en una partició de k vs N-k, viu realment en l'espai S(C^d ** \otimes k) * S(C^d ** \otimes N-k)
#La transposta en la base computacional tindrà diversos eigenvalues que seran 0, però no ens interessen!

dim = [d**biggest_bipartition,d**(N-biggest_bipartition)] #Dimensions dels espais de Hilbert de la bipartició més gran

IS_T = partial_transpose(IS, 0, dim) #Transposada Parcial de la identitat del subespai simètric respecte la partició més gran

min_eval_esperat = 1/math.comb(N,biggest_bipartition) #Mínim valor propi que trobem nosaltres

min_eval = np.linalg.eigvalsh(IS_T)[-symmetric_transpose_dimension] #Mínim valor propi que troba numpy en el subespai de S(C^d ** \otimes k) * S(C^d ** \otimes N-k)

print('Numèric', min_eval, '\n','Esperat', min_eval_esperat) #No hauria de dependre de d. També es pot buscar per internet o imprimir-te els passos intermitjos per veure què està passant

Numèric 0.5 
 Esperat 0.5


$$
\mathcal{S}(\mathbb{C}^{2}\otimes\mathbb{C}^{2})
$$

**Càlcul de les equacions que ens donin els bounds d'$\alpha$**

In [25]:
import sympy as sp
from sympy import I, Matrix, symbols, conjugate, sqrt
from sympy.matrices import eye

#Definim les variables de manera simbòlica
a, b, c = symbols('a b c', complex=True) #Igual també es pot provar amb valors reals
alpha = symbols('alpha', real=True)
constraint = sp.Eq(conjugate(a)*a + conjugate(b)*b + conjugate(c)*c, 1) #Forcem que estiguin normalitzats a,b,c. 

# Dickes (estaria més bé una funció per dimensions generals, però bueno)
D0 = Matrix([[1], [0], [0], [0]])  # |D0⟩ = |00⟩
D1 = Matrix([[0], [1/sqrt(2)], [1/sqrt(2)], [0]])  # |D1⟩ = (|01⟩ + |10⟩) / sqrt(2)
D2 = Matrix([[0], [0], [0], [1]])  # |D2⟩ = |11⟩

DS0 = Matrix([[1], [0], [0]])
DS1 = Matrix([[0], [1], [0]])
DS2 = Matrix([[0], [0], [1]])

# Define the pure state psi = a*|D0⟩ + b*|D1⟩ + c*|D2⟩
psi = a * DS0 + b * DS1 + c * DS2

# Symmetric identity operator for the symmetric subspace (3x3)
IdentitySymmetric = eye(3)

# Projection operator |psi⟩⟨psi| in the symmetric basis
KetBraPsi = psi * psi.H

DickeStates = [D0, D1, D2]

# Operator: I_Symmetric + alpha * |psi⟩⟨psi|
OperatorSym = IdentitySymmetric + alpha * KetBraPsi
print('Lambda(rho)_S')
sp.pprint(OperatorSym)

# Compute the eigenvalues of the operator
eigenvalues_sym = OperatorSym.eigenvals()

# Print the eigenvalues in the symmetric basis
print("Eigenvalues in symmetric basis:")
print(eigenvalues_sym)
# --- Step 2: Transform to the computational basis ---

OperatorComp = Matrix.zeros(4, 4)
for i in range(3):
    for j in range(3):
        # Add the contribution of each entry multiplied by |Di⟩⟨Dj|
        OperatorComp += OperatorSym[i, j] * (DickeStates[i] * DickeStates[j].H)

# Simplify the operator in the computational basis
#OperatorComp = OperatorComp.applyfunc(lambda x: sp.simplify(x))

# Display the operator in the computational basis
print("Operator in computational basis:")
sp.pprint(OperatorComp)


# Compute the eigenvalues in the computational basis
eigenvalues_comp = OperatorComp.eigenvals()

# Print the eigenvalues in the computational basis
print("\nEigenvalues in computational basis:")
print(eigenvalues_comp)

# --- Step 3: Perform the partial transpose with respect to the second qubit ---

def partial_transpose(matrix):
    """Performs the partial transpose with respect to the second qubit for a 4x4 SymPy matrix."""
    # Ensure the input is a 4x4 matrix
    if matrix.shape != (4, 4):
        raise ValueError("Input matrix must be 4x4.")
    
    # Initialize an empty 4x4 matrix for the result
    result = sp.Matrix.zeros(4, 4)
    
    # Define the indices for 2-qubit systems (each qubit has dimension 2)
    basis_indices = [(i, j) for i in range(4) for j in range(4)]
    
    # Map the 4x4 indices to 2x2x2x2 indices and perform the partial transpose
    for i, j in basis_indices:
        # Convert 1D indices to 2-qubit indices: i = (i1, i2), j = (j1, j2)
        i1, i2 = divmod(i, 2)
        j1, j2 = divmod(j, 2)
        
        # Partial transpose swaps the second qubit indices (i2 and j2)
        transposed_i = i1 * 2 + j2
        transposed_j = j1 * 2 + i2
        
        # Assign the value to the transposed position
        result[transposed_i, transposed_j] = matrix[i, j]
    
    return result


# Perform the partial transpose on the operator in the computational basis
OperatorPartialTranspose = partial_transpose(OperatorComp)
sp.pprint(OperatorPartialTranspose)

# Compute the eigenvalues of the partially transposed operator
eigenvalues_pt = OperatorPartialTranspose.eigenvals()

# Print the eigenvalues after partial transpose
print("\nEigenvalues after partial transpose:")
for eig in eigenvalues_pt:
    print(eig)


Lambda(rho)_S
⎡    _            _          _  ⎤
⎢a⋅α⋅a + 1    a⋅α⋅b      a⋅α⋅c  ⎥
⎢                               ⎥
⎢      _        _            _  ⎥
⎢  α⋅b⋅a    α⋅b⋅b + 1    α⋅b⋅c  ⎥
⎢                               ⎥
⎢      _          _        _    ⎥
⎣  α⋅c⋅a      α⋅c⋅b    α⋅c⋅c + 1⎦
Eigenvalues in symmetric basis:
{1: 2, a*alpha*conjugate(a) + alpha*b*conjugate(b) + alpha*c*conjugate(c) + 1: 1}
Operator in computational basis:
⎡                  _          _            ⎤
⎢    _      √2⋅a⋅α⋅b   √2⋅a⋅α⋅b         _  ⎥
⎢a⋅α⋅a + 1  ────────   ────────     a⋅α⋅c  ⎥
⎢              2          2                ⎥
⎢                                          ⎥
⎢       _       _          _             _ ⎥
⎢√2⋅α⋅b⋅a   α⋅b⋅b   1  α⋅b⋅b   1  √2⋅α⋅b⋅c ⎥
⎢────────   ───── + ─  ───── + ─  ──────── ⎥
⎢   2         2     2    2     2     2     ⎥
⎢                                          ⎥
⎢       _       _          _             _ ⎥
⎢√2⋅α⋅b⋅a   α⋅b⋅b   1  α⋅b⋅b   1  √2⋅α⋅b⋅c ⎥
⎢────────   ───── + ─  ────

In [3]:
#No calen ni la metitat, però vaig fent copy paste dels que sempre tinc
import numpy as np
import math
import sympy as sp
from scipy.stats import unitary_group
from scipy.stats import ortho_group
#import matplotlib.pyplot as plt
import cvxpy as cp
#import pypoman
from toqito.perms import symmetric_projection
from toqito.channels import partial_transpose
from qutip import Qobj, partial_transpose
import scipy.linalg as la
from qutip import partial_transpose as qutip_partial_transpose
from toqito.channels import partial_transpose as toqito_partial_transpose
from scipy.optimize import fmin_cobyla
import scipy.linalg as sp_linalg
import numpy as np
import math
from scipy.stats import ortho_group
from joblib import Parallel, delayed
import multiprocessing
from functools import partial
import copy

**Una altra manera de mirar l'eigenvalue mínim**

In [6]:
d = 2 #qudits
N = 3 #How many qudits
k=math.floor(N/2)
n = math.comb(N,int(math.floor(N/2)))
print('Binomial', 1/n)
D = d**N #Hilbert space dimension
dime = [d]*N
syst = [1]*k + [0]*(N-k)

IS = Qobj(symmetric_projection(dim=d,p_val=N))
IS.dims=[dime,dime]
IST = partial_transpose(IS, mask=syst)
IST = IST.full()
print(np.linalg.eigvalsh(IST)[-math.comb(d+k-1,k) * math.comb(d+(N-k)-1,N-k)])

Binomial 0.3333333333333333
0.3333333333333331


**Primer intent pel cas bipartit $\mathcal{S}(\mathbb{C}^{d}\otimes\mathbb{C}^{d})$**

In [8]:
#Generar el vector de Dicke states, només en C^d x C^d
def dicke_states(d):
    D= int(d**2)
    DS = int(d*(d+1)/2)

    Di=np.zeros((D,DS))
    k=0
    I=np.eye(d)

    for i in range(d):
        for j in range(i,d):
            Di[:,k]=np.kron(I[:,i],I[:,j])+np.kron(I[:,j],I[:,i])
            Di[:,k]=Di[:,k]/np.linalg.norm(Di[:,k])
            k=k+1
    
    return Di

#Funció per fer la partial transpose, només en C^d x C^d. Podria fer servir qualsevol llibreria, però no se perquè vaig mirar de fer-me-la jo.
def partial_transpose_manual(matrix, d):
    if matrix.shape != (d * d, d * d):
        raise ValueError(f"Matrix shape must be {d*d}x{d*d} for dimension d={d}.")
    reshaped_matrix = np.reshape(matrix, (d, d, d, d))
    partially_transposed = np.transpose(reshaped_matrix, (2, 1, 0, 3))
    return np.reshape(partially_transposed, (d * d, d * d))


**Brute Force (també brutalment ineficient, però per veure la idea del què estem fent en dimensions baixes, funciona prou bé)**

In [9]:
d= 2
alpha = -2/3-0.001 #El valor d'alpha que trobem a la demostració, només depèn de N, en principi! Cal mirar numèricament si podem o no podem fer-ho millor! 
#Directament només considero alpha_{-}! Però es pot mirar tot el mateix amb alpha_{+}
tries = 10000 #Amb quantes unitàries diferents ho volem provar. Convindria afegir una mesura de temps per tal de saber quant dura una iteració.
ks = tries / 10

N = 2
DS = int(d*(d+1)/2) #Dimensió del subespai simètric
D=d**N

hola=np.ones(DS)/(DS+alpha)
hola[0] = 1-(DS-1)/(DS+alpha) #Em creo els punts extrems

for k in range(tries):
    if k % ks == 0:
         print(k)
    u = ortho_group.rvs(DS) #Les matrius ortogonals no són les úniques unitàries, aquí estem perdent informació! Però a nivell de càlcul són algo més ràpides per dimensions elevades
    #Sino: scipy.stats.unitary_group.rvs(DS)
    rho=np.diag(hola) #Matriu diagonal dels valors propis
    rho = u @ rho @ u.conjugate().transpose() # u rho u^dagger
    sigma=np.zeros((D,D), dtype=complex)
    #Tornem a la base computacional fent servir els dicke states
    for i in range(DS):
            for j in range(DS):
                sigma += rho[i,j]*np.outer(dicke_states(d)[:,i],dicke_states(d)[:,j])

    #density_matrix = sigma

    #rho = Qobj(density_matrix, dims=[[d,d],[d,d]])

    #rho_0_Transpose = partial_transpose(rho, mask=[1,0], method='dense')

    #rho_0_Transpose_evals = np.linalg.eigvalsh(rho_0_Transpose.full())

    a=np.min(np.linalg.eigvals(partial_transpose_manual(sigma,d)))

    if a <= - 0.0000001: #El zero numèric he considerat sobre 10e-06, però és un valor bastant arbitrari
        print('NPT', np.real(a), k)
        break

0
1000
2000
3000
4000
5000
6000
7000
8000
9000


**Intent de minimització sobre totes les Us del valor mínim d'$\alpha$ pel qual ens mantenim en APPT** 

No és ni ràpid, ni eficient. És una primera prova per mirar si serveix i què i com canviar-ho en un futur!

En aquest cas, fem un barrido d'$\alpha$ i minimitzem sobre U a cada valor. Problema: Molt sovint no arribem al veritable minim del mínim valor propi.

És a dir: Per valors d'$\alpha$ pels quals l'estat no és APPT, la minimització no ens troba la U que el fa NPT -> No és prou bo!

In [10]:
def unitary_constraint_wrapper(DS):
    def constraint(U_flat):
        U_real = U_flat[:DS*DS].reshape((DS, DS))
        U_imag = U_flat[DS*DS:].reshape((DS, DS))
        U = U_real + 1j * U_imag

        identity = np.eye(DS)
        U_conj_T = U.conj().T
        unitary_diff = np.max(np.abs(U @ U_conj_T - identity))

        return 1e-6 - unitary_diff
    return constraint


# Objective function to minimize the minimal eigenvalue of the transformed matrix
def objective_func(U_flat,DS, dicke_state, rhooo, pt, dimens):
    U_real = U_flat[:DS*DS].reshape((DS, DS))
    U_imag = U_flat[DS*DS:].reshape((DS, DS))
    
    # Combine real and imaginary parts into a complex matrix
    U = U_real + 1j * U_imag
    
    # Project U onto the unitary group using polar decomposition
    U, _ = la.polar(U)
    
    # Compute the transformed matrix sigma = U @ rhooo @ U.T
    sigma = U @ rhooo @ U.conj().T

    matrix = np.zeros((D, D), dtype=complex)
    for i in range(DS):
        for j in range(DS):
            matrix += sigma[i, j] * np.outer(dicke_state[:,i], dicke_state[:,j])
    
    # Perform partial transpose on the first subsystem (implement this without qutip if needed)
    matrix_pt = toqito_partial_transpose(matrix, pt, dimens)
    min_eigval = np.min(np.linalg.eigvalsh(matrix_pt))

    return min_eigval  # Since we are minimizing, return the minimal eigenvalue directly


In [11]:
d=2
DS=int(d*(d+1)/2)
N=2 #De moment només ens funciona per estats bipartits!
D=d**N
array = [1,0]
masks_0=[array]
Nfloor = int(np.floor(N/2))
pt = list(range(Nfloor)) #Com volem fer la partial transpose
dimens=[d,d]

dicke_state = dicke_states(d) #Funció definida fa una estona per bipartits!

alphas=np.linspace(-0.5555555555555556,-0.6,10) #El rang d'alphas i la quantitat de punts s'han de modificar segons el moment i què es vulgui veure.
for alpha in alphas:
    print('\n', 'ALPHA', alpha)
    #Set the eigenvalues given a value of alpha
    vec=np.ones(DS)/(DS+alpha)
    vec[-1]=1-(DS-1)/(DS+alpha)

    rhooo = np.diag(vec)

    # Set initial guess for the optimization
    initial_guess = np.random.uniform(-1, 1, 2 * DS * DS)

    # Set constraints for the optimization (unitary constraint)
    constraints = [unitary_constraint_wrapper(DS)]

    # Perform the constrained optimization using fmin_cobyla
    objective = partial(objective_func, DS=DS, dicke_state=dicke_state, rhooo=rhooo, pt=pt, dimens=dimens)
    result = fmin_cobyla(objective, initial_guess, constraints, rhobeg=0.5, rhoend=1e-7, maxfun=5000)

    # Optimal U from the result
    U_real_opt = result[:DS*DS].reshape((DS, DS))
    U_imag_opt = result[DS*DS:].reshape((DS, DS))
    U_opt = U_real_opt + 1j * U_imag_opt
    #print("Optimal U matrix (unitary):", U_opt)

    # Project onto the unitary group using polar decomposition to ensure it
    U_opt, _ = la.polar(U_opt)

    # Check if U_opt is unitary
    identity = np.eye(DS)
    if not np.allclose(U_opt @ U_opt.conj().T, identity) or not np.allclose(U_opt.conj().T @ U_opt, identity):
        print("Final U_opt is not unitary!")
    else:
        print("Final U_opt is unitary.")

    # Compute and print eigenvalues
    sigma_opt = U_opt @ rhooo @ U_opt.conj().T
    matrix_opt = np.zeros((D, D), dtype=complex)
    for i in range(DS):
        for j in range(DS):
            matrix_opt += sigma_opt[i, j] * np.outer(dicke_state[:,i], dicke_state[:,j])

    # Partial transpose and eigenvalue check (implement your own if qutip is unavailable)
    matrix_pt_opt = toqito_partial_transpose(matrix_opt, pt , dimens)
    min_eigenvalue = np.min(np.linalg.eigvalsh(matrix_pt_opt))

    print("Minimal eigenvalue after optimization:", min_eigenvalue)


 ALPHA -0.5555555555555556
Final U_opt is unitary.
Minimal eigenvalue after optimization: 0.06146295134120468

 ALPHA -0.5604938271604939
Final U_opt is unitary.
Minimal eigenvalue after optimization: 0.06008201720610104

 ALPHA -0.5654320987654321
Final U_opt is unitary.
Minimal eigenvalue after optimization: 0.058692156380744744

 ALPHA -0.5703703703703704
Final U_opt is unitary.
Minimal eigenvalue after optimization: 0.05729333546436605

 ALPHA -0.5753086419753086
Final U_opt is unitary.
Minimal eigenvalue after optimization: 0.05588552074524751

 ALPHA -0.5802469135802469
Final U_opt is unitary.
Minimal eigenvalue after optimization: 0.05446867819503844

 ALPHA -0.5851851851851851
Final U_opt is unitary.
Minimal eigenvalue after optimization: 0.05304277346305572

 ALPHA -0.5901234567901235
Final U_opt is unitary.
Minimal eigenvalue after optimization: 0.051607771870582984

 ALPHA -0.5950617283950617
Final U_opt is unitary.
Minimal eigenvalue after optimization: 0.05016363840514473

En el cas bipartit, $\mathcal{S}(\mathbb{C}^{d}\otimes\mathbb{C}^{d})$, és més fàcil, ja que la transposada parcial viu en $\mathcal{S}(\mathbb{C}^{d}) \otimes \mathcal{S}(\mathbb{C}^{d})$ que és dimensionalment equivalent a $\mathbb{C}^{d}\otimes\mathbb{C}^{d}$, l'espai global. Llavors el que busquem és el punt on el mínim valor propi passa de positiu a negatiu (passant per $0$, en principi).

Tot i això, aquesta minimització no sempre ens troba el valor mínim real que podríem assolir!

**Multipartit**

Pel cas multipartit, lo anterior no passa. A més, ens cal adaptar les definicions dels dicke states i de les transposades parcials, ja que no són tant senzill som en el cas bipartit.

***Tripartit***

In [15]:
#Els hem de crear d'una manera diferent els Dicke States que en el cas bipartit! La funció anterior no servia.
def dicke_states(d):
    # D is the dimension of the full Hilbert space (d^3 for tripartite system)
    D = d**3
    
    # DS is the number of distinct symmetric Dicke states
    # This is based on the number of ways to distribute "excitations" across the three subsystems
    DS = int((d * (d + 1) * (d + 2)) // 6)  # Choose 3 from d + 2
    
    Di = np.zeros((D, DS))
    k = 0
    I = np.eye(d)

    # Loop through the possible indices for tripartite system
    for i in range(d):
        for j in range(i, d):
            for l in range(j, d):
                # Create symmetric combinations for tripartite system
                symmetric_state = (np.kron(np.kron(I[:, i], I[:, j]), I[:, l])
                                   + np.kron(np.kron(I[:, i], I[:, l]), I[:, j])
                                   + np.kron(np.kron(I[:, j], I[:, i]), I[:, l])
                                   + np.kron(np.kron(I[:, j], I[:, l]), I[:, i])
                                   + np.kron(np.kron(I[:, l], I[:, i]), I[:, j])
                                   + np.kron(np.kron(I[:, l], I[:, j]), I[:, i]))

                # Normalize the symmetric state
                Di[:, k] = symmetric_state / np.linalg.norm(symmetric_state)
                k += 1
    
    return Di

d=3
N=3 #Aquesta box és només per tripartit (3 qudits)
DS=math.comb(d+N-1,N)
D=d**N
Nfloor = int(np.floor(N/2))
pt = list(range(Nfloor))
dicke_state = dicke_states(d)
array = [1,0,0]
masks_0=[array]
dimens=[d,d,d]

dicke_states = dicke_states(d)

alphas=np.linspace(-0.55,-0.57,2) #De nou:Modificar segons la necessitat!
for alpha in alphas:
    print('\n', 'ALPHA', alpha)
    vec=np.ones(DS)/(DS+alpha)
    vec[-1]=1-(DS-1)/(DS+alpha)

    rhooo = np.diag(vec)

    # Set initial guess for the optimization
    initial_guess = np.random.uniform(-1, 1, 2 * DS * DS)

   # Define constraint wrapper for DS
    constraints = [unitary_constraint_wrapper(DS)]

    #Perform the constrained optimization
    objective = partial(objective_func, DS=DS, dicke_state=dicke_state, rhooo=rhooo, pt=pt, dimens=dimens)
    result = fmin_cobyla(objective, initial_guess, constraints, rhobeg=0.5, rhoend=1e-7, maxfun=5000)

    # Optimal U from the result
    U_real_opt = result[:DS*DS].reshape((DS, DS))
    U_imag_opt = result[DS*DS:].reshape((DS, DS))
    U_opt = U_real_opt + 1j * U_imag_opt
    #print("Optimal U matrix (unitary):", U_opt)

    # Project onto the unitary group using polar decomposition to ensure it
    U_opt, _ = la.polar(U_opt)

    # Check if U_opt is unitary
    identity = np.eye(DS)
    if not np.allclose(U_opt @ U_opt.conj().T, identity) or not np.allclose(U_opt.conj().T @ U_opt, identity):
        print("Final U_opt is not unitary!")
    else:
        print("Final U_opt is unitary.")

    # Compute and print eigenvalues
    sigma_opt = U_opt @ rhooo @ U_opt.conj().T
    matrix_opt = np.zeros((D, D), dtype=complex)
    for i in range(DS):
        for j in range(DS):
            matrix_opt += sigma_opt[i, j] * np.outer(dicke_state[:,i], dicke_state[:,j])

    # Partial transpose and eigenvalue check (implement your own if qutip is unavailable)
    matrix_pt_opt = toqito_partial_transpose(matrix_opt, pt , dimens)
    min_eigenvalue = np.min(np.linalg.eigvalsh(matrix_pt_opt))

    print("Minimal eigenvalue after optimization:", min_eigenvalue, '\n')



 ALPHA -0.55
Final U_opt is unitary.
Minimal eigenvalue after optimization: -5.111485997699539e-17 


 ALPHA -0.57
Final U_opt is unitary.
Minimal eigenvalue after optimization: -6.586186654191216e-17 



**Tetrapartit**

N=4 (Millor generalitzar la minimització $\forall d,N$)

In [35]:
def dicke_states_tetrapartite(d):
    # D is the dimension of the full Hilbert space (d^4 for tetrapartite system)
    D = d**4
    
    # DS is the number of distinct symmetric Dicke states for a tetrapartite system
    DS = int((d * (d + 1) * (d + 2) * (d + 3)) // 24)  # Choose 4 from d + 3
    
    Di = np.zeros((D, DS))
    k = 0
    I = np.eye(d)

    # Loop through the possible indices for tetrapartite system
    for i in range(d):
        for j in range(i, d):
            for l in range(j, d):
                for m in range(l, d):
                    # Create symmetric combinations for tetrapartite system
                    symmetric_state = (np.kron(np.kron(np.kron(I[:, i], I[:, j]), I[:, l]), I[:, m])
                                       + np.kron(np.kron(np.kron(I[:, i], I[:, j]), I[:, m]), I[:, l])
                                       + np.kron(np.kron(np.kron(I[:, i], I[:, l]), I[:, j]), I[:, m])
                                       + np.kron(np.kron(np.kron(I[:, i], I[:, l]), I[:, m]), I[:, j])
                                       + np.kron(np.kron(np.kron(I[:, i], I[:, m]), I[:, j]), I[:, l])
                                       + np.kron(np.kron(np.kron(I[:, i], I[:, m]), I[:, l]), I[:, j])
                                       + np.kron(np.kron(np.kron(I[:, j], I[:, i]), I[:, l]), I[:, m])
                                       + np.kron(np.kron(np.kron(I[:, j], I[:, i]), I[:, m]), I[:, l])
                                       + np.kron(np.kron(np.kron(I[:, j], I[:, l]), I[:, i]), I[:, m])
                                       + np.kron(np.kron(np.kron(I[:, j], I[:, l]), I[:, m]), I[:, i])
                                       + np.kron(np.kron(np.kron(I[:, j], I[:, m]), I[:, i]), I[:, l])
                                       + np.kron(np.kron(np.kron(I[:, j], I[:, m]), I[:, l]), I[:, i])
                                       + np.kron(np.kron(np.kron(I[:, l], I[:, i]), I[:, j]), I[:, m])
                                       + np.kron(np.kron(np.kron(I[:, l], I[:, i]), I[:, m]), I[:, j])
                                       + np.kron(np.kron(np.kron(I[:, l], I[:, j]), I[:, i]), I[:, m])
                                       + np.kron(np.kron(np.kron(I[:, l], I[:, j]), I[:, m]), I[:, i])
                                       + np.kron(np.kron(np.kron(I[:, l], I[:, m]), I[:, i]), I[:, j])
                                       + np.kron(np.kron(np.kron(I[:, l], I[:, m]), I[:, j]), I[:, i])
                                       + np.kron(np.kron(np.kron(I[:, m], I[:, i]), I[:, j]), I[:, l])
                                       + np.kron(np.kron(np.kron(I[:, m], I[:, i]), I[:, l]), I[:, j])
                                       + np.kron(np.kron(np.kron(I[:, m], I[:, j]), I[:, i]), I[:, l])
                                       + np.kron(np.kron(np.kron(I[:, m], I[:, j]), I[:, l]), I[:, i])
                                       + np.kron(np.kron(np.kron(I[:, m], I[:, l]), I[:, i]), I[:, j])
                                       + np.kron(np.kron(np.kron(I[:, m], I[:, l]), I[:, j]), I[:, i]))

                    # Normalize the symmetric state
                    Di[:, k] = symmetric_state / np.linalg.norm(symmetric_state)
                    k += 1
    
    return Di



Brute Force pero paralelitzant el càlcul de les $U$s i el mínim eigenvalue pot ser una solució?

In [39]:
# Function to generate random orthogonal matrix with a given random state
def random_unitary(DS, random_state):
    return ortho_group.rvs(DS, random_state=random_state)

# Function to perform one iteration of the loop
def compute_iteration(k_start, k_end, rho, dicke_states, d, N, chunk_size, seed):
    # Use the provided seed to create a RandomState for reproducibility
    random_state = np.random.RandomState(seed)
    
    DS = math.comb(d + N - 1, N)
    D = d ** N
    dimens = [d] * N
    counter = 0

    # Create a local copy of rho for each thread to ensure independent calculations
    local_rho = copy.deepcopy(rho)
    
    for k in range(k_start, k_end):
        # Increment counter
        counter += 1
        
        # Random unitary transformation with the random state provided
        u = random_unitary(DS, random_state)
        local_rho = u @ local_rho @ u.conjugate().transpose()
        sigma = np.zeros((D, D), dtype=complex)
        for i in range(DS):
            for j in range(DS):
                sigma += local_rho[i, j] * np.outer(dicke_states[:, i], dicke_states[:, j])
        # Perform partial transpose and compute minimum eigenvalue
        a = np.min(np.linalg.eigvals(toqito_partial_transpose(sigma, [0, 1], dimens)))

        # Print progress after every 10% of chunk_size
        if counter % (chunk_size // 10) == 0:
            print(f'Progress: {counter} tries in this chunk of size {chunk_size}')
        
        # Check if NPT condition is met
        if a <= -0.0000001:
            print(f'Alpha: {alpha}, Minimal negative value of a: {np.real(a)}')
            break
    return a

def parallel_processing(tries, num_workers):
    chunk_size = tries // num_workers

    results = Parallel(n_jobs=num_workers)(
        delayed(compute_iteration)(
            i * chunk_size, (i + 1) * chunk_size, rho, dicke_states, d, N, chunk_size, np.random.randint(1e9)
        )
        for i in range(num_workers)
    )